<font size="6">**Nathan GALMICHE and Victor PIRIOU**</font>

<font size="5">**1.2 Data visualisation**</font>

In [ ]:
from sklearn.linear_model import LinearRegression
from matplotlib import image as mpimg
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from skimage.transform import resize
import pickle
import numpy as np
import cv2
import os

In [ ]:
def read_paths(file_name):
    with open(file_name) as f:
        return np.array([line.rstrip("\n") for line in f])


train_img_path = read_paths("data/300w_train_images.txt")
train_landmark_paths = read_paths("data/300w_train_landmarks.txt")
test_img_path = read_paths("data/helen_testset.txt")
test_landmark_paths = read_paths("data/helen_testset_landmarks.txt")

fig = plt.figure(figsize=(15, 20))
selected_indices = np.random.choice(len(train_img_path), 12)

for i, line in enumerate(train_img_path[selected_indices]):
    pts = np.loadtxt("data/" + train_landmark_paths[selected_indices[i]])

    try:
        img = mpimg.imread("data/" + line)
    except Exception as e:
        print(f"Exception: {e}")

    ax = fig.add_subplot(3, 4, i + 1)
    ax.scatter(pts[:, 0], pts[:, 1], linewidths=0.5)
    ax.imshow(img)

<font size="5">**1.3 Data augmentation**</font>

In [ ]:
nb_selected_img = 1000

In [ ]:
# this cell doesn't need to be re-executed

# Check and create directory before the loop
if not os.path.exists("data/helen/preprocessed_train_img"):
    os.mkdir("data/helen/preprocessed_train_img")

for i, landmark in enumerate(train_landmark_paths):
    if i == nb_selected_img:
        break

    print(f"\r{i+1}/{min(len(train_landmark_paths), nb_selected_img)}", end="")

    img = mpimg.imread("data/" + train_img_path[i])
    pts = np.loadtxt("data/" + landmark)
    xmin, xmax = pts[:, 0].min(), pts[:, 0].max()
    ymin, ymax = pts[:, 1].min(), pts[:, 1].max()
    w, h = xmax - xmin, ymax - ymin

    # Widening of the bounding box with simplified boundary checks
    xmin_exp = max(int(xmin - w * 0.15), 0)
    xmax_exp = min(int(xmax + w * 0.15), len(img[0]) - 1)
    ymin_exp = max(int(ymin - h * 0.15), 0)
    ymax_exp = min(int(ymax + h * 0.15), len(img) - 1)

    img = img[ymin_exp:ymax_exp, xmin_exp:xmax_exp]
    h, w = img.shape[:2]
    img = resize(img, (128, 128))

    # Simplified newpts calculation using list comprehension
    newpts = [
        f"{(pt[0] - xmin_exp) / w * 128} {(pt[1] - ymin_exp) / h * 128}" for pt in pts
    ]

    with open(f"data/helen/preprocessed_train_img/figure{i}.txt", "w") as f:
        f.write("\n".join(newpts))

    plt.imsave(f"data/helen/preprocessed_train_img/figure{i}.png", img)

In [ ]:
# this cell doesn't need to be re-executed

for i, landmark in enumerate(test_landmark_paths):
    print(f"\r{i+1}/{len(test_landmark_paths)}", end="")

    img = mpimg.imread("data/" + test_img_path[i])
    pts = np.loadtxt("data/" + landmark)
    xmin, xmax = pts[:, 0].min(), pts[:, 0].max()
    ymin, ymax = pts[:, 1].min(), pts[:, 1].max()
    w, h = xmax - xmin, ymax - ymin

    # widening of the bounding box
    xmin_exp = max(int(xmin - w * 0.15), 0)
    xmax_exp = min(int(xmax + w * 0.15), img.shape[1] - 1)
    ymin_exp = max(int(ymin - h * 0.15), 0)
    ymax_exp = min(int(ymax + h * 0.15), img.shape[0] - 1)

    img = img[ymin_exp:ymax_exp, xmin_exp:xmax_exp]
    h, w = img.shape[:2]
    img = resize(img, (128, 128))

    newpts = [
        f"{(pt[0] - xmin_exp) / w * 128} {(pt[1] - ymin_exp) / h * 128}" for pt in pts
    ]

    os.makedirs("data/helen/preprocessed_test_img", exist_ok=True)
    with open(f"data/helen/preprocessed_test_img/figure{i}.txt", "w") as f:
        f.write("\n".join(newpts))

    plt.imsave(f"data/helen/preprocessed_test_img/figure{i}.png", img)

<font size="3">**Check that the preprocess went well:**</font>

In [ ]:
fig = figure(figsize=(15, 20))

selected_indices = np.random.choice(nb_selected_img, 12)

for cpt, i in enumerate(selected_indices):
    pts = np.loadtxt(f"data/helen/preprocessed_train_img/figure{i}.txt")
    img = mpimg.imread(f"data/helen/preprocessed_train_img/figure{i}.png")

    ax = fig.add_subplot(3, 4, cpt + 1)

    ax.scatter(pts[:, 0], pts[:, 1], linewidths=0.5)
    ax.imshow(img)

In [ ]:
# this cell doesn't need to be re-executed

fig = figure(figsize=(15, 20))

selected_indices = np.random.choice(len(test_landmark_paths), 12)

cpt = 0

for i in selected_indices:
    pts = np.loadtxt("data/helen/preprocessed_test_img/figure" + str(i) + ".txt")
    img = mpimg.imread("data/helen/preprocessed_test_img/figure" + str(i) + ".png")

    ax = fig.add_subplot(3, 4, cpt + 1)

    ax.scatter(pts[:, 0], pts[:, 1], linewidths=0.5)
    ax.imshow(img)

    cpt += 1

In [ ]:
def compute_mean_landmarks(dataset_type, landmark_paths, nb_selected_img):
    file_path = "data/helen/preprocessed_{}_img/figure".format(dataset_type)
    mean = np.loadtxt(file_path + "0.txt")
    limit = (
        min(len(landmark_paths), nb_selected_img)
        if dataset_type == "train"
        else len(landmark_paths)
    )

    for i in range(1, limit):
        print("\r{}/{}".format(i, limit), end="")
        pts = np.loadtxt(file_path + str(i) + ".txt")
        mean += pts

    mean /= limit
    return mean


def save_or_load_landmarks(dataset_type, landmark_paths, nb_selected_img):
    os.makedirs("backup", exist_ok=True)
    file_name = "backup/mean_{}_landmarks.pkl".format(dataset_type)
    if not os.path.exists(file_name):
        mean_landmarks = compute_mean_landmarks(
            dataset_type, landmark_paths, nb_selected_img
        )
        with open(file_name, "wb") as file:
            pickle.dump(mean_landmarks, file)
    else:
        with open(file_name, "rb") as file:
            mean_landmarks = pickle.load(file)
    return mean_landmarks


print("train:")
mean_train_landmarks = save_or_load_landmarks(
    "train", train_landmark_paths, nb_selected_img
)

print("\n\ntest:")
mean_test_landmarks = save_or_load_landmarks(
    "test", test_landmark_paths, nb_selected_img
)


def plot_landmarks(image_path, landmarks):
    plt.imshow(mpimg.imread(image_path))
    plt.scatter(landmarks[:, 0], landmarks[:, 1])
    plt.show()


plot_landmarks("data/helen/preprocessed_train_img/figure0.png", mean_train_landmarks)
plot_landmarks("data/helen/preprocessed_test_img/figure0.png", mean_test_landmarks)

<font size="3">**5. Why do we generate these perturbations? How could they be estimated automatically?**</font>

We generate these perturbations to match the variety of faces we can encounter. Also, pictures of faces can be taken from more or less far and with varying angles. The perturbations are data augmentations 

In [ ]:
# this cell doesn't need to be re-executed

num_images = min(len(train_landmark_paths), nb_selected_img)
for i in range(num_images):
    print(f"\r{i+1}/{num_images}", end="")

    img = mpimg.imread(f"data/helen/preprocessed_train_img/figure{i}.png")
    for j in range(10):
        newPts = mean_train_landmarks.copy()

        tx = np.random.randint(-20, 20)
        ty = np.random.randint(-20, 20)
        sx = np.random.randint(-20, 20) / 100
        sy = np.random.randint(-20, 20) / 100

        perturb = []
        for pt in newPts:
            pt[0] = min(max(pt[0] * (1 + sx) + tx, 0), 128)
            pt[1] = min(max(pt[1] * (1 + sy) + ty, 0), 128)
            perturb.append(f"{pt[0]} {pt[1]}")

        perturb_str = "\n".join(perturb)
        with open(f"data/helen/preprocessed_train_img/figure{i}_{j}.txt", "w") as f:
            f.write(perturb_str)

In [ ]:
# this cell doesn't need to be re-executed

plt.imshow(mpimg.imread("data/helen/preprocessed_train_img/figure498.png"))
pts = np.loadtxt("data/helen/preprocessed_train_img/figure498_6.txt")
plt.scatter(pts[:, 0], pts[:, 1])
plt.show()

<font size="5">**2.1   Feature extraction**</font>

<font size="3">**1. Why do we not directly use the raw value of the image pixels as a representation ?**</font>

Because we want a representation that is robust to small changes (e.g. small rotations) and perturbations (e.g. noise) applied on the image, SIFT descriptors are well suited. Indeed they are robust to translation, small rotation, illumination change and scale changes. They allow a spatial encoding.

<font size="3">**3. What is the dimension of each feature?**</font>

Each patch image is decomposed into 16 sub-regions of size $4 \times 4$ pixels. Each sub-region is encoded as a vector of size 8. Hence, the dimension of each feature is $4 \times 4 \times 8 = 128$.

<font size="3">**4. For each image, concatenate all the computed features from each landmarks. What is the
dimension of this feature vector?**</font>

We have 68 landmarks per image the concatenation of all the computed features results in vector of size $68 \times 128 = 8704$.

In [ ]:
def get_descriptors(image, keyPoints):
    sift = cv2.SIFT_create()
    _, descriptors = sift.compute(image, keyPoints)
    return descriptors


def process_landmarks(landmarks, paths, size, prefix, output_file):
    keyPoints = [cv2.KeyPoint(x, y, size) for x, y in landmarks]
    descriptors_list = []

    for i, path in enumerate(paths):
        print(f"\r{i+1}/{len(paths)}", end="")
        img = cv2.imread(f"{prefix}{path}.png")
        desc = get_descriptors(img, keyPoints).flatten()
        descriptors_list.append(desc)

    X = np.column_stack(descriptors_list)
    with open(output_file, "wb") as file:
        pickle.dump(X, file)
    return X


# TRAIN
print("train:")
if os.path.exists("backup/X0_train.pkl"):
    with open("backup/X0_train.pkl", "rb") as file:
        X0_train = pickle.load(file)
else:
    X0_train = process_landmarks(
        mean_train_landmarks,
        [f"figure{i}" for i in range(min(len(train_landmark_paths), nb_selected_img))],
        20,
        "data/helen/preprocessed_train_img/",
        "backup/X0_train.pkl",
    )

print("\n\nX0_train.shape =", X0_train.shape)

# TEST
print("\ntest:")
if os.path.exists("backup/X0_test.pkl"):
    with open("backup/X0_test.pkl", "rb") as file:
        X0_test = pickle.load(file)
else:
    X0_test = process_landmarks(
        mean_test_landmarks,
        [f"figure{i}" for i in range(len(test_landmark_paths))],
        16,
        "data/helen/preprocessed_test_img/",
        "backup/X0_test.pkl",
    )

print("\n\nX0_test.shape =", X0_test.shape)

<font size="5">**2.2   Dimensionality reduction**</font>

<font size="3">**1.What is the main interest of this dimensionality reduction? Could you cite some other
dimensionality reduction methods for machine learning?**</font>


The main interest of this dimensionality reduction is to reduce the computational cost of the approach.  
  
  However, when applying PCA we have to keep in mind that this method has strong assumptions. Indeed, it assumes that the features can be expressed by a linear combination of a few principal components that are orthogobal.  
  
Other dimensionality reduction methods can be decomposed in two mains categories :
1. **Methods that find a subset of the features.** These include *Forward selection, Backward elimination, Random Forests, etc.*
2. **Methods that project the features in a lower dimensional space.** This category can be decomposed into linear and non-linear methods. Linear methods include *Factor Analysis, Linear Discriminant Analysis, Truncated Singular Value Decomposition, etc.* Non linear methods include *Kernel PCA, t-distributed Stochastic Neighbor Embedding, Multidimensional Scaling and Isometric mapping.*

In [ ]:
def load_or_compute_pca(data_type, data):
    prefix = f"backup/{data_type}"
    files = [
        f"{prefix}_mean.pkl",
        f"{prefix}_eigenvectors.pkl",
        f"{prefix}_eigenvalues.pkl",
    ]

    if all(os.path.exists(file) for file in files):
        with open(files[0], "rb") as file:
            mean = pickle.load(file)
        with open(files[1], "rb") as file:
            eigenvectors = pickle.load(file)
        with open(files[2], "rb") as file:
            eigenvalues = pickle.load(file)
    else:
        mean = np.empty((0))
        mean, eigenvectors, eigenvalues = cv2.PCACompute2(
            data.transpose(), mean, retainedVariance=0.98
        )
        with open(files[0], "wb") as file:
            pickle.dump(mean, file)
        with open(files[1], "wb") as file:
            pickle.dump(eigenvectors, file)
        with open(files[2], "wb") as file:
            pickle.dump(eigenvalues, file)

    return mean, eigenvectors, eigenvalues


# Process both train and test data
print(f"\n{'train'}:")
train_mean, train_eigenvectors, train_eigenvalues = load_or_compute_pca(
    "train", X0_train
)
print("done")

print(f"\n{'test'}:")
test_mean, test_eigenvectors, test_eigenvalues = load_or_compute_pca("test", X0_test)
print("done")

In [ ]:
X0_tilde = np.matmul(train_eigenvectors, X0_train)
print("X0_tilde.shape:", X0_tilde.shape)

<font size="3">**3. What are the dimensions of the new resulting matrix $\tilde{\mathbf{X}}_0$ ?**</font>

We have $\tilde{\mathbf{X}}_0=\mathbf{A}_0 \mathbf{X}_0$ where $\mathbf{A}_0 \in \mathbb{R}^{M^{\prime} \times M}$ and $\mathbf{X}_0 \in \mathbb{R}^{M \times N}$ so $\tilde{\mathbf{X}}_0 \in \mathbb{R}^{M^{\prime} \times N}$.

Here $M^{\prime} = 218 $ is the smallest number of principal components that explained at least $98\%$ of the variance of the original data while $N = 1000$ is the number of images we chose.

<font size="5">**2.3   Displacement estimation**</font>

<font size="3">**2. Compute the prediction error (mean absolute error) on the training set.**</font>

In [ ]:
# we choose to not take into account the augmented data to make PCA computationnally tractable with our computers

backup_path = os.path.join("backup", "points_train.pkl")
if os.path.exists(backup_path):
    with open(backup_path, "rb") as file:
        points_train = pickle.load(file)
else:
    points_train = np.empty((136, 0))
    num_images = min(len(train_landmark_paths), nb_selected_img)
    for i in range(num_images):
        print(f"\r{i}/{num_images}", end="")
        pts = np.loadtxt(f"data/helen/preprocessed_train_img/figure{i}.txt").reshape(
            (136, 1)
        )
        points_train = np.append(points_train, pts, axis=1)

    with open(backup_path, "wb") as file:
        pickle.dump(points_train, file)

print("\ndone")

In [ ]:
mean_train_landmarks_ = np.tile(
    mean_train_landmarks.copy().reshape((136, 1)), (1, X0_tilde.shape[1])
)

X0_tilde = np.concatenate([X0_tilde, np.ones((1, X0_tilde.shape[1]))], axis=0)

delta = points_train - mean_train_landmarks_

reg = LinearRegression().fit(X0_tilde.T, delta.T)

print("done")

In [ ]:
train_pred = reg.predict(X0_tilde.T).T
print(
    f"'mean absolute error' sur l'ensemble d'apprentissage = {np.mean(np.abs(delta - train_pred))}"
)

In [ ]:
print(f"score sur l'ensemble d'apprentissage = {reg.score(X0_tilde.T, delta.T)}")

In [ ]:
train_pred = train_pred.reshape((-1, 68, 2))

<font size="3">**2. Display $s_0$ the
initial position of the model (in red) and $s_0 + \delta_s$ the displaced landmarks (in green) for
the first image of the training set.**</font>

In [ ]:
plt.imshow(mpimg.imread("data/helen/preprocessed_train_img/figure0.png"))
plt.scatter(
    mean_train_landmarks[:, 0], mean_train_landmarks[:, 1], color="red", linewidths=0.5
)
plt.scatter(
    mean_train_landmarks[:, 0] + train_pred[0, :, 0],
    mean_train_landmarks[:, 1] + train_pred[0, :, 1],
    color="green",
    linewidths=0.5,
)
plt.show()

<font size="3">**2. What can we conclude ?**</font>

The results are not good. It would eventually work better with another size of patch. 

<font size="3">**3. Why this prediction error is not relevant to evaluate our methods ?**</font>

This prediction is not relevant to evaluate our methods because it is biased. Indeed, it is computed from an image that was used to train the model. We have to use a test set.

<font size="3">**4. Compute the prediction error on the test set and display s0 the initial position of the
model (in red) and s0 + δs the displaced landmarks (in green) for the 5 first images of the
test set.**</font>

In [ ]:
# we choose to not take into account the augmented data to make PCA computationnally tractable with our computersif(os.path.exists('backup/points_test.pkl')):
if os.path.exists("backup/points_test.pkl"):
    with open("backup/points_test.pkl", "rb") as file:
        points_test = pickle.load(file)
else:
    points_test = np.array(
        [
            np.loadtxt(f"data/helen/preprocessed_test_img/figure{i}.txt").reshape(
                (136, 1)
            )
            for i in range(len(test_landmark_paths))
        ]
    ).reshape(136, -1)

    with open("backup/points_test.pkl", "wb") as file:
        pickle.dump(points_test, file)

In [ ]:
X0_tilde_test = np.matmul(test_eigenvectors, X0_test)

mean_test_landmarks_ = np.repeat(
    mean_test_landmarks.reshape((136, 1)), X0_tilde_test.shape[1], axis=1
)

X0_tilde_test = np.concatenate(
    [X0_tilde_test, np.ones((1, X0_tilde_test.shape[1]))], axis=0
)

delta_test = points_test - mean_test_landmarks_

reg_test = LinearRegression().fit(X0_tilde_test.T, delta_test.T)

In [ ]:
mae = np.mean(np.abs(delta_test - reg_test.predict(X0_tilde_test.T).T))
print(f"mean absolute error on the train set = {mae}")

In [ ]:
score = reg_test.score(X0_tilde_test.T, delta_test.T)
print(f"score on the test set = {score}")

In [ ]:
for i in range(5):
    plt.imshow(
        mpimg.imread("data/helen/preprocessed_test_img/figure" + str(i) + ".png")
    )
    plt.scatter(
        mean_train_landmarks[:, 0],
        mean_train_landmarks[:, 1],
        color="red",
        linewidths=0.5,
    )
    plt.scatter(
        mean_train_landmarks[:, 0] + train_pred[i, :, 0],
        mean_train_landmarks[:, 1] + train_pred[i, :, 1],
        color="green",
        linewidths=0.5,
    )
    plt.show()

<font size="3">**4. What can we conclude ?**</font>

The results are not good. It would probably work better with more data. In order to improve the performance we could also fully implement the cascade regression.